In [1]:
import numpy as np
import pandas as pd
import seaborn, time

from pomegranate import BayesianNetwork

ModuleNotFoundError: No module named 'pomegranate'

In [2]:
seaborn.set_style('whitegrid')

In [3]:
"""
X = np.random.randint(2, size=(2000, 7))
X[:,3] = X[:,1]
X[:,6] = X[:,1]

X[:,0] = X[:,2]

X[:,4] = X[:,5]

model = BayesianNetwork.from_samples(X, algorithm='exact')
print(model.structure)
model.plot()
"""

"\nX = np.random.randint(2, size=(2000, 7))\nX[:,3] = X[:,1]\nX[:,6] = X[:,1]\n\nX[:,0] = X[:,2]\n\nX[:,4] = X[:,5]\n\nmodel = BayesianNetwork.from_samples(X, algorithm='exact')\nprint(model.structure)\nmodel.plot()\n"

In [4]:
def sanitize_celeb_attr_data(unsanitize_attrs_file):
    
    attr_list = []
    
    unsanitize_attrs = open(unsanitize_attrs_file,'r').readlines()
    
    for a in unsanitize_attrs:
        attr_line = a.strip('\n').split(' ')
        attrs_sanitized = [attr for attr in attr_line if attr != '']
        attr_list.append(attrs_sanitized)
        
    with open('/home/c01muya/vae-disentanglement/datasets/celeba/list_attr_celeba_sanitized.txt','w') as list_attr_celeb_sen:
        for attr_line in attr_list:
            list_attr_celeb_sen.write(
                ",".join(attr_line) + "\n"
            )

In [5]:
#sanitize_celeb_attr_data("/home/c01muya/vae-disentanglement/datasets/celeba/list_attr_celeba.txt")

In [40]:
celeb_attrs = pd.read_csv("/home/c01muya/vae-disentanglement/datasets/celeba/list_attr_celeba_sanitized.txt", 
                          sep=",", encoding='ascii')
celeb_attrs.drop('Image_ID', axis='columns', inplace=True)
celeb_attrs.head()

,5_o_Clock_Shadow,Arched_Eyebrows,Attractive,Bags_Under_Eyes,Bald,Bangs,Big_Lips,Big_Nose,Black_Hair,Blond_Hair,...,Sideburns,Smiling,Straight_Hair,Wavy_Hair,Wearing_Earrings,Wearing_Hat,Wearing_Lipstick,Wearing_Necklace,Wearing_Necktie,Young
0,-1,1,1,-1,-1,-1,-1,-1,-1,-1,...,-1,1,1,-1,1,-1,1,-1,-1,1
1,-1,-1,-1,1,-1,-1,-1,1,-1,-1,...,-1,1,-1,-1,-1,-1,-1,-1,-1,1
2,-1,-1,-1,-1,-1,-1,1,-1,-1,-1,...,-1,-1,-1,1,-1,-1,-1,-1,-1,1
3,-1,-1,1,-1,-1,-1,-1,-1,-1,-1,...,-1,-1,1,-1,1,-1,1,1,-1,1
4,-1,1,1,-1,-1,-1,1,-1,-1,-1,...,-1,-1,-1,-1,-1,-1,1,-1,-1,1


In [41]:

hair_color_conditions = [
    celeb_attrs['Black_Hair'] == 1,
    celeb_attrs['Brown_Hair'] == 1,
    celeb_attrs['Blond_Hair'] == 1,
    celeb_attrs['Gray_Hair']  == 1,
    (celeb_attrs['Black_Hair'] != 1) & (celeb_attrs['Brown_Hair'] != 1) & (celeb_attrs['Blond_Hair'] != 1) & (celeb_attrs['Gray_Hair']  != 1)
]

hair_color_values = ['Black', 'Brown', 'Blond', 'Gray', 'Unknown']
celeb_attrs['Hair_Color'] = np.select(hair_color_conditions, hair_color_values)
#celeb_attrs[ ['Hair_Color','Black_Hair', 'Brown_Hair', 'Blond_Hair', 'Gray_Hair'] ]
celeb_attrs.drop(['5_o_Clock_Shadow', 'Black_Hair','Brown_Hair','Blond_Hair','Gray_Hair'], axis='columns', inplace=True)

In [42]:
hair_type_conditions = [
    celeb_attrs['Bald'] == 1,
    celeb_attrs['Bangs'] == 1,
    celeb_attrs['Straight_Hair'] == 1,
    celeb_attrs['Wavy_Hair']  == 1,
    (celeb_attrs['Bald'] != 1) & (celeb_attrs['Bangs'] != 1) & (celeb_attrs['Straight_Hair'] != 1) & (celeb_attrs['Wavy_Hair'] != 1)
]

hair_type_values = ['Bald', 'Bangs', 'Straight', 'Wavy','Unknown']

celeb_attrs['Hair_Type'] = np.select(hair_type_conditions, hair_type_values)
#celeb_attrs[ ['Hair_Type','Bald', 'Bangs', 'Straight_Hair', 'Wavy_Hair'] ]
celeb_attrs.drop(['Bald', 'Bangs', 'Straight_Hair', 'Wavy_Hair'], axis='columns', inplace=True)

In [44]:
celeb_attrs.head(10)

,Arched_Eyebrows,Attractive,Bags_Under_Eyes,Big_Lips,Big_Nose,Blurry,Bushy_Eyebrows,Chubby,Double_Chin,Eyeglasses,...,Sideburns,Smiling,Wearing_Earrings,Wearing_Hat,Wearing_Lipstick,Wearing_Necklace,Wearing_Necktie,Young,Hair_Color,Hair_Type
0,1,1,-1,-1,-1,-1,-1,-1,-1,-1,...,-1,1,1,-1,1,-1,-1,1,Brown,Straight
1,-1,-1,1,-1,1,-1,-1,-1,-1,-1,...,-1,1,-1,-1,-1,-1,-1,1,Brown,Unknown
2,-1,-1,-1,1,-1,1,-1,-1,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,1,Unknown,Wavy
3,-1,1,-1,-1,-1,-1,-1,-1,-1,-1,...,-1,-1,1,-1,1,1,-1,1,Unknown,Straight
4,1,1,-1,1,-1,-1,-1,-1,-1,-1,...,-1,-1,-1,-1,1,-1,-1,1,Unknown,Unknown
5,1,1,-1,1,-1,-1,-1,-1,-1,-1,...,-1,-1,1,-1,1,-1,-1,1,Brown,Wavy
6,-1,1,1,1,1,-1,1,-1,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,1,Black,Straight
7,1,-1,1,1,-1,-1,-1,-1,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,1,Black,Unknown
8,1,1,-1,1,-1,-1,-1,-1,-1,-1,...,-1,1,1,-1,1,-1,-1,1,Unknown,Bangs
9,-1,1,-1,-1,-1,-1,-1,-1,-1,-1,...,-1,-1,-1,-1,1,-1,-1,1,Unknown,Wavy


In [45]:
celeb_attrs_mat = celeb_attrs.to_numpy()
print(celeb_attrs_mat.shape)

(202599, 33)


In [46]:
celeb_attrs_mat_subset = celeb_attrs_mat[:10000,:]
print(celeb_attrs_mat_subset.shape)

(10000, 33)


In [47]:
model = BayesianNetwork.from_samples(celeb_attrs_mat_subset, 
                                     state_names=celeb_attrs.columns.to_list(), 
                                     algorithm='chow-liu')
print(model.structure)

((), (27,), (4,), (0,), (13,), (1,), (31,), (4,), (7,), (1,), (17,), (27,), (27,), (27,), (24,), (17,), (14,), (13,), (12,), (12,), (11,), (31,), (11,), (17,), (12,), (13,), (31,), (0,), (13,), (13,), (1,), (13,), (27,))


In [49]:
#%matplotlib inline

model.plot("../reports/learned-dag-3.pdf")

In [ ]:
# 1. The labels in celeba don't capture the variation within each attribute value. Not all blond hair at the same for example. 
# So we shouldn't use just a categorical to represent that information.


In [1]:
import torch

In [5]:
torch.cuda.list_gpu_processes()

'pynvml module not found, please install pynvml'

In [3]:
dir(torch.cuda)

['Any',
 'BFloat16Storage',
 'BFloat16Tensor',
 'BoolStorage',
 'BoolTensor',
 'ByteStorage',
 'ByteTensor',
 'CharStorage',
 'CharTensor',
 'ComplexDoubleStorage',
 'ComplexFloatStorage',
 'CudaError',
 'DeferredCudaCallError',
 'Device',
 'Dict',
 'DoubleStorage',
 'DoubleTensor',
 'Event',
 'FloatStorage',
 'FloatTensor',
 'HalfStorage',
 'HalfTensor',
 'IntStorage',
 'IntTensor',
 'List',
 'LongStorage',
 'LongTensor',
 'Optional',
 'ShortStorage',
 'ShortTensor',
 'Stream',
 'Tuple',
 'Union',
 '_CudaBase',
 '_CudaDeviceProperties',
 '_Graph',
 '_StorageBase',
 '__annotations__',
 '__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__path__',
 '__spec__',
 '_check_capability',
 '_check_cubins',
 '_cudart',
 '_device',
 '_device_t',
 '_dummy_type',
 '_get_device_index',
 '_initialization_lock',
 '_initialized',
 '_is_in_bad_fork',
 '_lazy_call',
 '_lazy_init',
 '_lazy_new',
 '_queued_calls',
 '_sleep',
 '_tls',
 '_utils',
 'amp',
 '

In [7]:
!nvidia-smi

Sat May  7 17:19:39 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.51.06    Driver Version: 450.51.06    CUDA Version: 11.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  A100-SXM4-40GB      On   | 00000000:07:00.0 Off |                    0 |
| N/A   33C    P0    78W / 400W |   3320MiB / 40537MiB |     38%      Default |
|                               |                      |             Disabled |
+-------------------------------+----------------------+----------------------+
|   1  A100-SXM4-40GB      On   | 00000000:0F:00.0 Off |                    0 |
| N/A   

In [2]:
adj_mat = [(), (27,), (4,), (0,), (13,), (1,), (31,), (4,), (7,), (1,), 
           (17,), (27,), (27,), (27,), (24,), (17,), (14,), (13,), (12,), 
           (12,), (11,), (31,), (11,), (17,), (12,), (13,), (31,), (0,), 
           (13,), (13,), (1,), (13,), (27,)]



In [3]:
t1 = ()
t2= (2,2)

In [5]:
len(t1)

0

In [21]:
def find_top_level_nodes(adj_mat):
    
    first_layer = []
    
    for node_idx, node in enumerate(adj_mat):
        # in in-degree is 0 add it to first layer node
        if len(node) == 0:
            first_layer.append(node_idx)
    
    return None if len(first_layer) == 0 else first_layer  

def find_child_nodes(parent_nodes, adj_mat):

    child_nodes = []
    
    for node_idx, node in enumerate(adj_mat):
        for p in parent_nodes:
            if p in node:
                child_nodes.append(node_idx)

    return None if len(child_nodes) == 0 else child_nodes  

def list_layers(adj_mat):
    first_layer = find_top_level_nodes(adj_mat)
    children = []
    parents = first_layer
    while children is not None:
        children = find_child_nodes(parents, adj_mat)
        print(children)
        parents = children

def get_layer_mask(parents, children, adj_mat):
    
    """
    In mask, rows correspond to parents and columns correspond to children
    """
    P, C = len(parents), len(children)
    
    # initialize with zeros
    mask = np.zeros(shape=(P,C))
    
    for i, p in enumerate(parents): 
        for j, c in enumerate(children):
            # check DAG interaction and fill-in 1's where interaction is justified according to given DAG
            # if p's index is present in c's parent list
            if p in adj_mat[c]:
                mask[i, j] = 1.0
    
    return mask

#first_layer = find_top_level_nodes(adj_mat)
#second_layer = find_child_nodes(first_layer, adj_mat)  
#print(first_layer)
#print(second_layer)

list_layers(adj_mat)

[3, 27]
[1, 11, 12, 13, 32]
[4, 5, 9, 17, 18, 19, 20, 22, 24, 25, 28, 29, 30, 31]
[2, 6, 7, 10, 14, 15, 21, 23, 26]
[8, 16]
None


In [31]:
parents = [3, 27]
children = [1, 11, 12, 13, 32]
get_layer_mask(parents, children, adj_mat)

array([[0., 0., 0., 0., 0.],
       [1., 1., 1., 1., 1.]])

In [32]:
parents = [1, 11, 12, 13, 32]
children = [4, 5, 9, 17, 18, 19, 20, 22, 24, 25, 28, 29, 30, 31]
get_layer_mask(parents, children, adj_mat)

array([[0., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 0., 0., 1., 1., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 1., 0., 0., 1., 0., 0., 0., 0., 0.],
       [1., 0., 0., 1., 0., 0., 0., 0., 0., 1., 1., 1., 0., 1.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]])

In [26]:
Z = np.zeros(shape=(3,4))

In [29]:
for i in range(Z.shape[0]): 
    for j in range(Z.shape[1]):
        if i==j:
            Z[i,j] = 3
            
print(Z)

[[3. 0. 0. 0.]
 [0. 3. 0. 0.]
 [0. 0. 3. 0.]]


In [27]:
Z.shape 

(3, 4)

In [33]:
import torch



In [34]:
t = torch.Tensor([1195.6053, 1193.2589])
                 

In [35]:
nt = torch.cat(t, dim=1)
print(nt)

TypeError: cat() received an invalid combination of arguments - got (Tensor, dim=int), but expected one of:
 * (tuple of Tensors tensors, name dim, *, Tensor out)
 * (tuple of Tensors tensors, int dim, *, Tensor out)
